# Travaux Pratiques 2

Ce deuxième TP vous permettra de manipuler les méthodes d'imputation présentées dans la vidéo du module 2, à savoir :
* l'imputation par la moyenne,
* l'imputation par plus proches voisins,
* l'imputation itérative.

Le premier exercice est le plus théorique et permet de développer une intuition du fonctionnement des méthodes. Les exercices 2 et 3 sont plus pratiques.

**Note :** une imputation peut avoir plusieurs objectifs. Ici, vous étudierez des méthodes ayant pour but de minimiser une erreur d'imputation, en choisissant toujours les valeurs les plus probables. C'est un but tout à fait légitime, mais ces méthodes ont l'inconvénient de déformer la distribution des données, notamment en réduisant la variance. Elles ne sont donc pas indiquées lorsqu'on cherche à estimer la distribution.

# Importation des librairies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Librairies importées dans la solution

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ParameterGrid
from sklearn.datasets import load_breast_cancer

import warnings

# Exercice 1 : applications basiques des méthodes d'imputation

Dans cet exercice, vous utiliserez des données synthétiques en deux dimensions, afin de faciliter la visualisation des résultats.

## Question 1 : échantillon bivarié gaussien

Générez un échantillon bivarié (*i.e.* à $d=2$ variables), gaussien,

$$
\left( X_{i.} \right)_{1\leq i\leq n}
=\left( X_{i0}, X_{i1} \right)_{1\leq i\leq n}
$$

de taille $n=500$, de moyenne $\mu$ et de matrice de covariance $\Sigma$, en notant :

$$
\mu = \begin{bmatrix}
  \mu_0 \\[6pt] \mu_1
\end{bmatrix}, \quad
\Sigma = \begin{bmatrix}
  \sigma_0^2 & \rho \sigma_0 \sigma_1 \\[6pt]
  \rho \sigma_0 \sigma_1 & \sigma_1^2
\end{bmatrix},
$$

avec les valeurs suivantes :
$$
\mu_0 = 0,~\mu_1 = 0,~\sigma_0 = 1,~\sigma_1 = 0.7,~\rho = 0.8
$$

Vous enregistrerez l'échantillon dans une variable `xfull`.

Représentez l'échantillon à l'aide d'un nuage de points.

### Solution

In [ ]:
np.random.seed(0)

n = 500
d = 2
mu0 = 0.
mu1 = 0.
sig0 = 1.
sig1 = 0.7
rho = 0.8

mean = np.array([mu0, mu1])
cov = np.array([
    [sig0 ** 2, rho * sig0 * sig1],
    [rho * sig0 * sig1, sig1 ** 2]
    ])

xfull = np.random.multivariate_normal(mean, cov, size=n)

ax = sns.scatterplot(x=xfull[:, 0], y=xfull[:, 1], color=['#d1e5f0'])

## Question 2 : génération de valeurs manquantes

### Question 2a

Exécutez la cellule suivante pour générer les valeurs manquantes comme dans le TP1, et obtenir le jeu de données amputé, `xmiss`. Quel est le mécanisme de données manquantes ici ? Que représente la variable `p` ? Combien y a-t-il de *patterns* possibles, c'est-à-dire de cas de figure de NA possibles pour une ligne ?

In [ ]:
p = 0.4

xmiss = np.copy(xfull)
for j in range(d):
  miss_id = (np.random.uniform(0, 1, size=n) < p)
  xmiss[miss_id, j] = np.nan

display(pd.DataFrame(xmiss).head(10))

### Solution

Le mécanisme est MCAR car le manque de données, `miss_id`, est indépendant de `xfull`.

`p` représente la probabilité que chaque variable soit manquante (notons qu'on pourrait choisir un `p_j` différent pour chaque variable et rester MCAR).

Il y a ici 4 patterns :
* $X_0,~X_1$ observés,
* $X_0$ seul manquant,
* $X_1$ seul manquant,
* $X_0,~X_1$ manquants.

### Question 2b

Exécutez la cellule suivante pour représenter `xmiss`. Interprétez le graphique : que signifie chaque groupe de points ?

In [ ]:
where_full = ~np.isnan(xmiss[:, 0]) & ~np.isnan(xmiss[:, 1])
where_na0 = np.isnan(xmiss[:, 0]) & ~np.isnan(xmiss[:, 1])
where_na1 = np.isnan(xmiss[:, 1]) & ~np.isnan(xmiss[:, 0])
where_na01 = np.isnan(xmiss[:, 0]) & np.isnan(xmiss[:, 1])

ax = sns.scatterplot(x=xmiss[where_full, 0], y=xmiss[where_full, 1], color=['#d1e5f0'], label="?")

(xmin, xmax), (ymin, ymax) = ax.get_xlim(), ax.get_ylim()

_ = sns.scatterplot(x=xmin, y=xmiss[where_na0, 1], color=['#2194ac'], ax=ax, clip_on=False, label="?")
_ = sns.scatterplot(x=xmiss[where_na1, 0], y=ymin, color=['#2138ac'], ax=ax, clip_on=False, label="?")
_ = sns.scatterplot(x=[xmin], y=[ymin], color=['#ac6721'], ax=ax, clip_on=False, label="?")

_ = ax.set_xlim(xmin, xmax)
_ = ax.set_ylim(ymin, ymax)

_ = ax.set_xlabel(r'$X_0$')
_ = ax.set_ylabel(r'$X_1$')

ax.legend();

### Solution

Dans le premier groupe, les variables $X_{.0}$ et $X_{.1}$ sont observées, les points sont donc représentées normalement.

Dans le deuxième groupe, la variable $X_{.0}$ est manquante, on n'a accès qu'à la variable $X_{.1}$. Dans ce cas, les points sont donc représentés sur l'axe vertical, puisqu'on ne sait pas les placer horizontalement.

Inversement, dans le troisième groupe, la variable $X_{.1}$ est manquante, donc les points sont représentés sur l'axe horizontal car on ne sait pas les placer verticalement.

Finalement, les points du dernier groupe, pour lesquels les deux variables sont manquantes sont représentés dans le coin inférieur gauche.

Ces quatre cas de figure correspondent aux quatre *patterns* possibles.

## Question 3 : imputation par les moyennes

L'imputation la plus simple possible est l'imputation par les moyennes. En utilisant le module `sklearn.impute`, remplacez les valeurs manquantes de `xmiss` par la moyenne dans chaque variable, et représentez le jeu de données imputé en vous inspirant du graphique de la question 2b.

### Solution

In [ ]:
mean_imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
ximp_mean = mean_imputer.fit_transform(xmiss)

In [ ]:
ax = sns.scatterplot(x=xmiss[where_full, 0], y=xmiss[where_full, 1], color=['#d1e5f0'], label="Complet")

(xmin, xmax), (ymin, ymax) = ax.get_xlim(), ax.get_ylim()

_ = sns.scatterplot(x=ximp_mean[where_na0, 0], y=ximp_mean[where_na0, 1], color=['#2194ac'], ax=ax, clip_on=False, label=r"$X_0$ imputé")
_ = sns.scatterplot(x=ximp_mean[where_na1, 0], y=ximp_mean[where_na1, 1], color=['#2138ac'], ax=ax, clip_on=False, label=r"$X_1$ imputé")
_ = sns.scatterplot(x=ximp_mean[where_na01, 0], y=ximp_mean[where_na01, 1], color=['#ac6721'], ax=ax, clip_on=False, label=r"$X_0,X_1$ imputés")

ax.set_xlim(xmin, xmax);
ax.set_ylim(ymin, ymax);

_ = ax.set_xlabel(r'$X_0$')
_ = ax.set_ylabel(r'$X_1$')

ax.legend();

## Question 4 : minimisation de l'erreur d'imputation théorique * (question difficile nécessitant de connaître la notion d'espérance conditionnelle)

Pour un point $X_{i.} = \left( X_{i0}, X_{i1} \right)$ de l'échantillon (la donnée réelle sans valeur manquante), on définit son erreur quadratique moyenne d'imputation $E_i$ par:

$$
E_i
= \lVert X_{i.} - \hat X_{i.} \rVert^2
= \left(X_{i0} - \hat X_{i0} \right)^2
  + \left(X_{i1} - \hat X_{i1} \right)^2
$$

où $\hat X_{i.}$ est la donnée imputée.

L'erreur quadratique moyenne d'imputation $E$ sur l'échantillon est donnée par:

$$E = \frac{1}{n} \sum_{i=1}^{n} E_i$$

### Question 4a

Calculez l'erreur quadratique moyenne (MSE) de l'imputation par la moyenne de la question 3.

### Solution

In [ ]:
def mse(x_imp, x_true):
  n = len(x_true)
  return (1 / n) * np.sum((x_imp - x_true) ** 2)

In [ ]:
print(mse(ximp_mean, xfull))

### Question 4b

Dans ce cas bivarié, comment se simplifie l'erreur $E_i$ en fonction du pattern à la ligne $i$, c'est-à-dire en fonction de quelles variables sont manquantes ?

### Solution

$$
\begin{align*}
E_i = \left\{
\begin{array}{ll}
0 & \textrm{si tout observé} \\
\left(X_{i0} - \hat X_{i0} \right)^2 & \textrm{si }X_{i0}\textrm{ seul manquante} \\
\left(X_{i1} - \hat X_{i1} \right)^2 & \textrm{si }X_{i1}\textrm{ seul manquante} \\
\left(X_{i0} - \hat X_{i0} \right)^2 + \left(X_{i1} - \hat X_{i1} \right)^2 & \textrm{si }X_{i0}, X_{i1}\textrm{ manquantes}
\end{array}
\right.
\end{align*}
$$

### Question 4c

Supposons qu'on cherche à minimiser l'erreur quadratique moyenne d'imputation.

Dans ce modèle gaussien, à quel problème bien connu se ramène-t-on dans chaque pattern ? Quelle est l'imputation optimale dans chaque pattern ? Exprimez-la en fonction de $\mu_0$, $\mu_1$, $\sigma_0$, $\sigma_1$, $\rho$.

### Solution

Ce problème de minimisation des moindres carrés dans chaque pattern a une solution connue qui est l'espérance conditionnelle.

Lorsque $X_{.1}$ est la seule variable manquante:
$$
\mathbb{E}[X_{.1} \mid X_{.0} = x_{.0}] = \mu_1 + \rho \frac{\sigma_1}{\sigma_0} (x_{.0} - \mu_0)
$$
Lorsque $X_{.0}$ est la seule variable manquante:
$$
\mathbb{E}[X_{.0} \mid X_{.1} = x_{.1}] = \mu_0 + \rho \frac{\sigma_0}{\sigma_1} (x_{.1} - \mu_1)
$$
Lorsque $X_{.0}$ et $X_{.1}$ sont manquantes:
$$
\mathbb{E}[X_{.0}, X_{.1}] = \left(\mu_0,~\mu_1\right).
$$

### Question 4d

Implémentez cette imputation dans une fonction. Calculez son erreur quadratique.

### Solution

In [ ]:
def conditional_expectation_imputation(xmiss, mu0, mu1, sig0, sig1, rho):
    mask = np.isnan(xmiss)
    # get integer patterns from the 2D boolean mask
    # (0,0) -> 0; (1,0) -> 1; (0,1) -> 2; (1,1) -> 3
    patterns = mask[:, 0].astype(int) + 2 * mask[:, 1].astype(int)  # shape: (N,)

    impx0 = np.c_[mu0 + rho * sig0 / sig1 * (xmiss[:, 1] - mu1), xmiss[:, 1]]
    impx1 = np.c_[xmiss[:, 0], mu1 + rho * sig1 / sig0 * (xmiss[:, 0] - mu0)]
    impx01 = np.c_[np.full_like(xmiss[:, 0], mu0), np.full_like(xmiss[:, 1], mu1)]

    # stack impputation cases into shape (4, N, 2)
    imputations = np.stack([xmiss, impx0, impx1, impx01], axis=0)  # shape: (4, N, 2)

    # select appropriate rows from each imputation case using the patterns
    rows = np.arange(len(patterns))  # shape: (500,)

    imp = imputations[patterns, rows]  # shape: (500, 2)

    return imp

In [ ]:
ximp_ce = conditional_expectation_imputation(xmiss, mu0, mu1, sig0, sig1, rho)
print(mse(ximp_ce, xfull))

### Question 4e

Toujours en vous inspirant du graphique de la question 2b, représentez l'imputation par l'espérance conditionnelle.

### Solution

In [ ]:
ax = sns.scatterplot(x=xmiss[where_full, 0], y=xmiss[where_full, 1], color=['#d1e5f0'], label="Complet")

(xmin, xmax), (ymin, ymax) = ax.get_xlim(), ax.get_ylim()

_ = sns.scatterplot(
    x=ximp_ce[where_na0, 0], y=ximp_ce[where_na0, 1],
    color=['#2194ac'], ax=ax, clip_on=False, label=r"$X_0$ imputé")
_ = sns.scatterplot(
    x=ximp_ce[where_na1, 0], y=ximp_ce[where_na1, 1],
    color=['#2138ac'], ax=ax, clip_on=False, label=r"$X_1$ imputé")
_ = sns.scatterplot(
    x=ximp_ce[where_na01, 0], y=ximp_ce[where_na01, 1],
    color=['#ac6721'], ax=ax, clip_on=False, label=r"$X_0,X_1$ imputés")

ax.set_xlim(xmin, xmax);
ax.set_ylim(ymin, ymax);

_ = ax.set_xlabel(r'$X_0$')
_ = ax.set_ylabel(r'$X_1$')

ax.legend();

**Remarque:** l'objet de cette question théorique était, en plus de manipuler la notion de pattern, de montrer que même dans un cas gaussien, l'imputation idéale ne consiste pas à tout ramener à une seule et unique droite de régression. Il y a une expression par pattern !

## Question 5 : imputation itérative avec régressions linéaires

En pratique, on ne connaît bien sûr pas $\mu$ et $\Sigma$, on ne sait même pas si les données sont gaussiennes. Il faut donc réaliser des régressions linéaires pour estimer les paramètres de la distribution. On peut le faire de manière itérative avec la classe `IterativeImputer` du module `sklearn.impute`, en utilisant une régression linéaire comme estimateur de base. Implémentez cette imputation, et représentez-la de la même manière que précédemment. Calculez également sa MSE.

### Solution

In [ ]:
imputer = IterativeImputer(estimator=LinearRegression())
ximp_lr = imputer.fit_transform(xmiss)

In [ ]:
print(mse(ximp_lr, xfull))

In [ ]:
ax = sns.scatterplot(x=xmiss[where_full, 0], y=xmiss[where_full, 1], color=['#d1e5f0'])

(xmin, xmax), (ymin, ymax) = ax.get_xlim(), ax.get_ylim()

_ = sns.scatterplot(x=ximp_lr[where_na0, 0], y=ximp_lr[where_na0, 1], color=['#2194ac'], ax=ax, clip_on=False)
_ = sns.scatterplot(x=ximp_lr[where_na1, 0], y=ximp_lr[where_na1, 1], color=['#2138ac'], ax=ax, clip_on=False)
_ = sns.scatterplot(x=ximp_lr[where_na01, 0], y=ximp_lr[where_na01, 1], color=['#ac6721'], ax=ax, clip_on=False)

ax.set_xlim(xmin, xmax);
ax.set_ylim(ymin, ymax);

## Question 6 : plus proches voisins

Le modèle linéaire gaussien ne convient pas toujours aux données qu'on étudie en pratique. Dans le cas général, les méthodes non paramétriques sont plus flexibles. Un exemple essentiel d'imputation non paramétrique est l'imputation par plus proches voisins.

Implémentez cette imputation à l'aide de `sklearn.impute`, calculez la MSE et représentez l'imputation sur un nuage de points.

L'hyperparamètre le plus important est le nombre de voisins à utiliser, pour trouver un équilibre biais-variance : comparez plusieurs valeurs, en MSE et graphiquement.

### Solution

In [ ]:
imputer = KNNImputer(n_neighbors=10)
ximp_knn = imputer.fit_transform(xmiss)

In [ ]:
print(mse(ximp_knn, xfull))

In [ ]:
ax = sns.scatterplot(x=xmiss[where_full, 0], y=xmiss[where_full, 1], color=['#d1e5f0'])

(xmin, xmax), (ymin, ymax) = ax.get_xlim(), ax.get_ylim()

_ = sns.scatterplot(x=ximp_knn[where_na0, 0], y=ximp_knn[where_na0, 1], color=['#2194ac'], ax=ax, clip_on=False)
_ = sns.scatterplot(x=ximp_knn[where_na1, 0], y=ximp_knn[where_na1, 1], color=['#2138ac'], ax=ax, clip_on=False)
_ = sns.scatterplot(x=ximp_knn[where_na01, 0], y=ximp_knn[where_na01, 1], color=['#ac6721'], ax=ax, clip_on=False)

ax.set_xlim(xmin, xmax);
ax.set_ylim(ymin, ymax);

## Question 7 : forêts aléatoires

Un autre exemple important d'imputation non paramétrique est l'imputation itérative avec forêts aléatoires ou un autre modèle à base d'arbres de décision. Utilisez à nouveau `IterativeImputer` pour imputer le jeu de données, cette fois-ci avec une forêt aléatoire comme estimateur de base. Calculez sa MSE et représentez-la graphiquement.

Les hyperparamètres les plus importants à régler sont d'abord ceux de l'estimateur de base : pour la forêt aléatoire, c'est le nombre d'estimateurs et la profondeur maximale des arbres. Pour l'imputation itérative, il peut être intéressant de limiter la profondeur des arbres pour plus de stabilité. Dans `IterativeImputer`, réglez également le nombre d'itérations maximal `max_iter` et le critère d'arrêt `tol`.

Attention, avec de jeux de données de grande dimension (beaucoup d'observations, beaucoup de variables), cette méthode est coûteuse en temps de calcul. Vous pourrez ajuster les hyperparamètres `n_nearest_features` et `skip_complete` pour réduire ce coût.

Notons que `IterativeImputer` est encore en phase exérimentale et son API pourra être amenée à changer.

### Solution

In [ ]:
imputer = IterativeImputer(estimator=RandomForestRegressor(n_estimators=10, max_depth=3), max_iter=10, tol=0.001)
ximp_rf = imputer.fit_transform(xmiss)

In [ ]:
print(mse(ximp_rf, xfull))

In [ ]:
ax = sns.scatterplot(x=xmiss[where_full, 0], y=xmiss[where_full, 1], color=['#d1e5f0'])

(xmin, xmax), (ymin, ymax) = ax.get_xlim(), ax.get_ylim()

_ = sns.scatterplot(x=ximp_rf[where_na0, 0], y=ximp_rf[where_na0, 1], color=['#2194ac'], ax=ax, clip_on=False)
_ = sns.scatterplot(x=ximp_rf[where_na1, 0], y=ximp_rf[where_na1, 1], color=['#2138ac'], ax=ax, clip_on=False)
_ = sns.scatterplot(x=ximp_rf[where_na01, 0], y=ximp_rf[where_na01, 1], color=['#ac6721'], ax=ax, clip_on=False)

ax.set_xlim(xmin, xmax);
ax.set_ylim(ymin, ymax);

# Exercice 2: imputation itérative avec forêts aléatoires

Dans cet exercice, vous recoderez à la main l'algorithme d'imputation itérative, à partir de l'estimateur de base `RandomForestRegressor`. Vous définirez une fonction, en partant de la cellule suivante, qui prend en entrée le jeu de données incomplet, et renvoie le jeu imputé.

On implémente en fait l'algorithme `missforest`, accessible dans [Steckhoven et al. (2012), Algorithm 1](https://academic.oup.com/bioinformatics/article/28/1/112/219101).

In [ ]:
def impute_manualrandomforest(xmiss):
    x_imputed = ...
    return x_imputed

## Question 1

Dans la fonction `impute_manualrandomforest`, créez une variable booléenne `mask` indiquant où sont les valeurs manquantes dans `xmiss`.

Ensuite, déterminez l'ordre des colonnes de`xmiss` par taux de `NA` croissant et enregistrez le résultat dans une variable `order` (ici il n'y a que 2 colonnes, mais vous implémenterez l'algorithme général).

### Solution

In [ ]:
def impute_manualrandomforest(xmiss):
    mask = np.isnan(xmiss)
    # get order of columns by increasing number of nans
    order = np.argsort(np.isnan(xmiss).sum(axis=0))

    x_imputed = ...
    return x_imputed

## Question 2

L'initialisation de l'algorithme est une imputation simple, par la moyenne. Dans la fonction, utilisez `SimpleImputer` pour imputer `xmiss` par ses moyennes, en enregistrant le résultat dans la variable `x_imputed`. Testez la fonction.

### Solution

In [ ]:
def impute_manualrandomforest(xmiss):
    mask = np.isnan(xmiss)
    # get order of columns by increasing number of nans
    order = np.argsort(np.isnan(xmiss).sum(axis=0))

    # impute the array by its means
    mean_imputer = SimpleImputer(strategy='mean')
    x_imputed = mean_imputer.fit_transform(xmiss)

    return x_imputed

In [ ]:
ximp_manrf = impute_manualrandomforest(xmiss)
print(mse(ximp_manrf, xfull))

## Question 3

Grâce à l'ordre établi à la question 1, sélectionnez la colonne avec le moins de valeurs manquantes, `col`.

En n'utilisant que les indices `i` où `xmiss[i, col]` est observée (donc où `mask[i, col]==0`), utilisez le jeu déjà imputé `x_imputed` pour entraîner une forêt aléatoire `rf` à prédire `x_imputed[:, col]` à partir de toutes les autres variables.

Utilisez ensuite `rf` pour prédire les valeurs manquantes de `xmiss[i, col]`. Ces nouvelles prédictions viennent remplacer, dans `x_imputed`, la première imputation naïve.

Indication : ici il n'y a que 2 colonnes. Pour entraîner la forêt avec une seule colonne, la fonction suivante pourra vous être utile.

In [ ]:
def ensure_2d_column(x):
    x = np.asarray(x)
    if x.ndim == 1:
        return x.reshape(-1, 1)  # (N,) → (N, 1)
    return x

### Solution

In [ ]:
def impute_manualrandomforest(xmiss):
    mask = np.isnan(xmiss)
    # get order of columns by increasing number of nans
    order = np.argsort(np.isnan(xmiss).sum(axis=0))

    # impute the array by its means
    mean_imputer = SimpleImputer(strategy='mean')
    x_imputed = mean_imputer.fit_transform(xmiss)

    # select column to impute
    col = order[0]
    other_cols = [c for c in order if c != col]

    # select indices where col is observed
    obs_indices = (mask[:, col] == 0)
    mis_indices = (mask[:, col] == 1)

    # fit random forest
    rf = RandomForestRegressor(n_estimators=10, max_depth=3)
    rf.fit(X=ensure_2d_column(x_imputed[obs_indices, other_cols]), y=x_imputed[obs_indices, col])
    # get new prediction
    pred = rf.predict(X=ensure_2d_column(x_imputed[mis_indices, other_cols]))
    # replace in array
    x_imputed[mis_indices, col] = pred

    return x_imputed

In [ ]:
ximp_manrf = impute_manualrandomforest(xmiss)
print(mse(ximp_manrf, xfull))

## Question 4

L'algorithme consiste à répéter l'étape précédente en bouclant sur toutes les colonnes, autant de fois qu'il est nécessaire jusqu'à ce que le critère d'arrêt soit vérifié ou que le nombre maximal de boucles soit atteint.

Le critère d'arrêt est vérifié lorsque la différence entre deux imputations successives est inférieure à un seuil. La fonction de différence est fournie ci-dessus.

In [ ]:
def difference(x_new, x_old):
  return np.sum((x_new - x_old) ** 2) / np.sum(x_new ** 2)

### Solution

In [ ]:
def impute_manualrandomforest(xmiss, max_iter=10, tol=0.001):
    mask = np.isnan(xmiss)
    # get order of columns by increasing number of nans
    order = np.argsort(np.isnan(xmiss).sum(axis=0))

    # impute the array by its means
    mean_imputer = SimpleImputer(strategy='mean')
    x_imputed = mean_imputer.fit_transform(xmiss)

    # no more than max_iter loops
    for iteration in range(max_iter):
        # loop over columns to impute
        for col in order:
            # save a copy to measure the difference between 2 successive imputations
            x_imputed_old = np.copy(x_imputed)

            other_cols = [c for c in order if c != col]

            # select indices where col is observed
            obs_indices = (mask[:, col] == 0)
            mis_indices = (mask[:, col] == 1)

            # fit random forest
            rf = RandomForestRegressor(n_estimators=10, max_depth=3)
            rf.fit(X=ensure_2d_column(x_imputed[obs_indices, other_cols]), y=x_imputed[obs_indices, col])
            # get new prediction
            pred = rf.predict(X=ensure_2d_column(x_imputed[mis_indices, other_cols]))
            # replace in array
            x_imputed[mis_indices, col] = pred

            diff = difference(x_imputed, x_imputed_old)
            if diff < tol:
                return x_imputed

    warnings.warn("max_iter was reached.")
    return x_imputed

In [ ]:
ximp_manrf = impute_manualrandomforest(xmiss)
print(mse(ximp_manrf, xfull))

# Exercice 3 : comparaison des méthodes d'imputation dans un jeu de données réel

Dans cet exercice, vous considérez le même jeu de données réel complet *Breast Cancer Wisconsin* que dans le TP1 (exercice 4). On génère 30% de valeurs manquantes de type MCAR.

In [ ]:
data = load_breast_cancer()
xfull = data['data']  # covariates, without missing values
diagnosis = data['target']  # target variable to predict, when the learning task is prediction
features_names = data['feature_names']

In [ ]:
pd.DataFrame(xfull, columns=features_names).head()

In [ ]:
n, d = xfull.shape  # data dimension

In [ ]:
np.random.seed(123)

p = 0.3
xmiss = np.copy(xfull)
for j in range(d):
  miss_id = np.random.uniform(0, 1, size=n) < p
  xmiss[miss_id, j] = np.nan

## Question 1

Appliquez les méthodes d'imputation vues dans l'exercice 1 à ce jeu de données, en optimisant les hyperparamètres :
* imputation par la moyenne
* imputation par plus proches voisins
* imputation itérative à base de régression linéaire
* imputation itérative à base de forêt aléatoire

Comparez leurs MSE : quelle est la meilleure méthode pour ce jeu de données ?

### Solution

In [ ]:
def evaluate_imputer(imputer, xfull, xmiss):
    ximp = imputer.fit_transform(xmiss)
    score = mse(ximp, xfull)
    print(f"{imputer.__str__():<60}: MSE = {score:.6f}")
    return score

def compare_imputers(xfull, xmiss):
    results = {}

    # 1. SimpleImputer (mean)
    simple = SimpleImputer(strategy='mean')
    score = evaluate_imputer(simple, xfull, xmiss)
    results['SimpleImputer (mean)'] = score

    # 2. KNNImputer (optimize k)
    for k in [3, 5, 10, 15]:
        knn = KNNImputer(n_neighbors=k)
        score = evaluate_imputer(knn, xfull, xmiss)
        results[f'KNNImputer (best_k={k})'] = score

    # 3. IterativeImputer + LinearRegression (optimize tol)
    for tol in [0.1, 0.01, 0.001, 0.0001]:
        iter_lr = IterativeImputer(estimator=LinearRegression(), tol=tol)
        score = evaluate_imputer(iter_lr, xfull, xmiss)
        results[f'IterativeImputer + LR (tol={tol})'] = score

    # 4. IterativeImputer + RandomForestRegressor (optimize depth, tol)
    param_grid = ParameterGrid({
        'max_depth': [3, 5, 7],
        'tol': [0.1, 0.01]
    })
    for params in param_grid:
        rf = RandomForestRegressor(n_estimators=10, max_depth=params['max_depth'], random_state=0)
        iter_rf = IterativeImputer(estimator=rf, max_iter=10, tol=params['tol'])
        score = evaluate_imputer(iter_rf, xfull, xmiss)
        key = f"IterativeImputer + RF (depth={params['max_depth']}, tol={params['tol']})"
        results[key] = score

    return results

In [ ]:
results = compare_imputers(xfull, xmiss)

In [ ]:
for name, score in sorted(results.items(), key=lambda x: x[1]):
    print(f"{name:<45}: MSE = {score:.6f}")

Parmi les modèles testés, la meilleure imputation est réalisée par l'imputation itérative. Le choix de l'estimateur de base, ainsi que le choix des hyperparamètres ne semble pas prépondérant, ou alors devrait être soumis à une procédure de sélection de modèles plus poussée.

La faible performance de l'imputation par plus proches voisins s'explique très probablement par la grande dimension.